<a href="https://colab.research.google.com/github/Modequick/Machine-Learning-Project/blob/main/Skripsi_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
!pip install kmodes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score



#new
from kmodes.kprototypes import KPrototypes


np.seterr(all='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [36]:
dataset_df = pd.read_csv("/content/drive/MyDrive/Skripsi/Change_Accom_Behavior_Hashed 2.csv")

df = dataset_df
dataset_df.head()

<ipython-input-36-1791d0b64d6a>:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_df = pd.read_csv("/content/drive/MyDrive/Skripsi/Change_Accom_Behavior_Hashed 2.csv")


,profile_id,order_id,accom_id,room_id,accom_chain_name,selling_price_idr,gbv_idr,coupon_idr,sales_date_utc7,sales_timestamp,...,next_category_1,next_category_2,next_category_3,next_latest_sentiment_analysis,next_room_type,next_bed_type,next_room_size_inmeter,count_buy_that_day,count_buy_that_month,count_buy_that_next_day
0,ET0LCvUlRKbtFLscAFkCi/WWLLw=,HCcNAiyXSymMGRQkSqzAAtFlMxc=,Uz+MOvMFoXZMCkmixPFGgpTm9lU=,L6H56k4Rou+RiVAeX2QFwrv6xwY=,NaN,524927.868189,570579.672607,0,2018-09-22,2018-09-22 09:35:41.027 UTC,...,NaN,NaN,NaN,NaN,DELUXE,ONE_DOUBLE_BED,NaN,2,7,1
1,exoQ3kUraPU4wFlzPkk5+ykBwTs=,sRDgi4iW3och3ZkFvI/7aW1P3xY=,PMJIaKNhV02UjxNiev3Ltce2kNQ=,h10O5X/RJjDGNa+4oSVmxZBhplQ=,iCwYKECg554FmWkZKOS6O7I8nXA=,853068.342720,721837.932454,0,2018-09-22,2018-09-22 03:16:16.417 UTC,...,NaN,NaN,NaN,NaN,NaN,KING,25.0,2,4,1
2,9GuIvtbdJFVdHFM/0x1p345wCwM=,xlSmSKB3tahvfYZ1DEq0Zu3nek0=,Y4UBm0bDkNkWK2JWpzmbJTlQLqc=,lqSNqRT0DKCxUC60BE3jFG5BwNQ=,NaN,192613.180745,188957.123586,0,2018-09-22,2018-09-22 01:28:14.882 UTC,...,NaN,NaN,NaN,NaN,NaN,ONE_DOUBLE_BED,18.5,2,4,1
3,QXB7Zobjy0aW6Q8SYt8DQD7fSrQ=,3d4xj1dEU8Q1tSY2APAxmcjlwFw=,TxJe7ItZLbtw1K8Aqlem51jNvaE=,P+HWuys1viVCcm4/aqWuEJ91tb0=,cCnyuGlSTM26W1cOgCNhFiEe+R8=,300377.277202,404480.886678,0,2018-09-22,2018-09-22 22:09:32.201 UTC,...,NaN,NaN,NaN,NaN,NaN,ONE_DOUBLE_BED_OR_TWO_SINGLE_B,26.0,2,4,1
4,UVEV2qykZPJQ/kfd20M2/86qKrM=,Qnf6/o5tErU5iMEiwzkWhFfiIjc=,kneU7W4rjhdyr4nKoVOP17bfaVk=,wmCmvcyYOxvDtr5DADgjj48ClEs=,HIwm7tZAAnF5sNurgy85MraVTB0=,225738.474167,231019.601478,0,2018-09-22,2018-09-22 15:08:32.967 UTC,...,NaN,NaN,NaN,NaN,DOUBLE,ONE_DOUBLE_BED,16.0,2,9,1


In [37]:
df["room_size_inmeter"].fillna(df["room_size_inmeter"].mean(),inplace = True)
df["next_room_size_inmeter"].fillna(df["next_room_size_inmeter"].mean(), inplace = True)
df["bed_type"].fillna(df["bed_type"].mode().iloc[0], inplace = True)
df["next_bed_type"].fillna(df["next_bed_type"].mode().iloc[0], inplace = True)
df["hotel_score"].fillna(df["hotel_score"].mean(),inplace = True)
df["next_hotel_score"].fillna(df["next_hotel_score"].mean(),inplace = True)

### Data Cleaning

In [38]:
missing_value = dataset_df.isnull().sum() *100 / len(dataset_df) 

df_missing_value = pd.DataFrame( {"missing_count" : dataset_df.isnull().sum(),
             "percent_missing" : missing_value })
print(df_missing_value.sort_values(by = "missing_count",ascending=False).head(20))
print("length of columns are : ", len(dataset_df.columns))

                                missing_count  percent_missing
customer_type                           18970       100.000000
next_latest_sentiment_analysis          18970       100.000000
latest_sentiment_analysis               18969        99.994729
next_category_3                         18946        99.873484
next_category_0                         18899        99.625725
next_category_1                         18899        99.625725
next_category_2                         18899        99.625725
category_3                              18854        99.388508
category_2                              18712        98.639958
category_1                              18712        98.639958
category_0                              18711        98.634686
pid_score                               14678        77.374802
next_pid_score                          14467        76.262520
room_type                               13324        70.237217
next_room_type                          12949        68

In [39]:
#selecting not from the list 
df_clean = dataset_df.loc[:, ~dataset_df.columns.isin(df_missing_value.sort_values(by = "missing_count",ascending=False).head(17).index)]
df_clean = df_clean.dropna()


df_clean = df_clean.loc[:, ~df_clean.columns.isin(["count_buy_that_day",
                                                   "count_buy_that_month",
                                                   "count_buy_that_next_day",
                                                   "room_id",
                                                   "order_id",
                                                   "profile_id",
                                                   "interface",
                                                   "last_issued_date",
                                                   "next_bed_type",
                                                   "month",
                                                   "next_time_stamp",
                                                   "sales_timestamp",
                                                   "next_is_use_coupon",
                                                   "next_num_rooms",
                                                   "next_gbv_idr",
                                                   "accom_id" ,
                                                   "check_in_date",
                                                   "check_out_date",
                                                   "sales_date_utc7",
                                                   "next_check_in_date",
                                                   "next_check_out_date",
                                                   "next_sales_date",
                                                   "next_is_use_coupon",
                                                   "is_use_coupon",
                                                   "next_coupon_idr",
                                                   "coupon_idr"])]



In [40]:
df = df_clean 
missing_value = df_clean.isnull().sum() *100 / len(dataset_df) 
df_missing_value = pd.DataFrame( {"missing_count" : df_clean.isnull().sum(),
             "percent_missing" : missing_value} )
print("length columns : ",len(df_clean.columns)  )
print(df_missing_value.sort_values(by = "missing_count",ascending=False))

length columns :  16
                        missing_count  percent_missing
selling_price_idr                   0              0.0
gbv_idr                             0              0.0
accom_region                        0              0.0
accom_star_rating                   0              0.0
num_rooms                           0              0.0
accom_type                          0              0.0
bed_type                            0              0.0
room_size_inmeter                   0              0.0
hotel_score                         0              0.0
next_accom_id                       0              0.0
next_accom_region                   0              0.0
next_accom_star_rating              0              0.0
next_accom_type                     0              0.0
next_selling_price                  0              0.0
next_hotel_score                    0              0.0
next_room_size_inmeter              0              0.0


In [41]:
df

,selling_price_idr,gbv_idr,accom_region,accom_star_rating,num_rooms,accom_type,bed_type,room_size_inmeter,hotel_score,next_accom_id,next_accom_region,next_accom_star_rating,next_accom_type,next_selling_price,next_hotel_score,next_room_size_inmeter
0,5.249279e+05,5.705797e+05,Jawa Barat,0,1,RESORT,ONE_DOUBLE_BED_OR_TWO_SINGLE_B,22.000000,8.742894,pmSgKDam2G81GGvziqaLZ1uu8uI=,Jawa Barat,3,HOTEL,6.198739e+05,7.395015,24.99785
1,8.530683e+05,7.218379e+05,Jawa Tengah,3,2,HOTEL,TWIN,16.000000,8.655503,5GeTZvyDUlmdvqPtk82Cgja7UTM=,Jawa Tengah,4,HOTEL,5.758693e+05,8.904430,25.00000
2,1.926132e+05,1.889571e+05,Jakarta,2,1,HOTEL,ONE_SINGLE_BED,12.000000,7.758292,FI1+e7bUFpzOX1jBaUNnyRpENYM=,Jakarta,2,HOTEL,2.179384e+05,7.909794,18.50000
3,3.003773e+05,4.044809e+05,Lampung Province,2,1,HOTEL,ONE_DOUBLE_BED,20.000000,8.686632,eGbvfr1SoxiZI4umM3lKK/YRGqQ=,Lampung Province,3,HOTEL,2.532418e+05,8.541579,26.00000
4,2.257385e+05,2.310196e+05,Aceh,1,1,HOMESTAY,TWIN,12.000000,8.257246,TQAbN/D9xL2fSELvW0UnubNuuy4=,Aceh,1,HOTEL,3.027667e+05,7.046263,16.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18965,2.527474e+05,2.227864e+05,Jawa Timur,2,1,APARTMENT,QUEEN,21.000000,7.517755,qdnFPc1h/femzNuyiOERDAT3eNw=,Jawa Timur,3,HOTEL,3.266155e+05,8.411149,18.00000
18966,1.074395e+06,1.071035e+06,Jawa Barat,4,1,HOTEL,ONE_KING_BED,44.000000,8.848756,dU8JmZhUw59kPNnNKrC1OtMIUFs=,Jawa Barat,4,HOTEL,8.707199e+05,8.622112,32.00000
18967,2.119371e+05,1.597081e+05,Kepulauan Riau,1,1,HOSTEL,ONE_DOUBLE_BED,9.000000,8.437692,0yOuAnmYUxpwyDuHKlgGuFMa0Qk=,Kepulauan Riau,3,HOTEL,4.097967e+05,8.335431,20.00000
18968,1.052873e+06,8.006005e+05,Bali,4,1,RESORT,ONE_QUEEN,26.082299,8.602368,OoIilhp++JHv3Ry0XDXTGpfbfSE=,Bali,0,GUESTHOUSE,1.157299e+05,8.501856,25.00000


In [52]:
x =["selling_price_idr",
    "gbv_idr",
    "accom_region",
    "accom_star_rating",
    "num_rooms",
    "accom_type",
    "bed_type"
    "room_size_inmeter",
    "hotel_score"
    ]
y =["next_accom_id",
    "next_accom_region",
    "next_accom_star_rating",
    "next_accom_type",
    "next_selling_price",
    "next_hotel_score",
    "next_room_size_inmeter"
    ]



In [55]:
df[y]

,next_accom_region,next_accom_star_rating,next_accom_type,next_selling_price,next_hotel_score,next_room_size_inmeter
0,Jawa Barat,3,HOTEL,6.198739e+05,7.395015,24.99785
1,Jawa Tengah,4,HOTEL,5.758693e+05,8.904430,25.00000
2,Jakarta,2,HOTEL,2.179384e+05,7.909794,18.50000
3,Lampung Province,3,HOTEL,2.532418e+05,8.541579,26.00000
4,Aceh,1,HOTEL,3.027667e+05,7.046263,16.00000
...,...,...,...,...,...,...
18965,Jawa Timur,3,HOTEL,3.266155e+05,8.411149,18.00000
18966,Jawa Barat,4,HOTEL,8.707199e+05,8.622112,32.00000
18967,Kepulauan Riau,3,HOTEL,4.097967e+05,8.335431,20.00000
18968,Bali,0,GUESTHOUSE,1.157299e+05,8.501856,25.00000


In [63]:
y_grouped = df.groupby(by = ["next_accom_id","next_accom_region","next_accom_type"],as_index = False)["next_accom_star_rating","next_selling_price","next_hotel_score","next_room_size_inmeter"].mean()

y = y_grouped.iloc[:,1:]
y.head()

<ipython-input-63-292ba41a2085>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  y_grouped = df.groupby(by = ["next_accom_id","next_accom_region","next_accom_type"],as_index = False)["next_accom_star_rating","next_selling_price","next_hotel_score","next_room_size_inmeter"].mean()


,next_accom_region,next_accom_type,next_accom_star_rating,next_selling_price,next_hotel_score,next_room_size_inmeter
0,Jawa Timur,HOMESTAY,1.0,896201.625996,8.434579,12.00000
1,Jawa Barat,HOTEL,2.0,513298.441428,8.420212,16.00000
2,Trang,HOTEL,2.0,294486.253722,8.831921,24.99785
3,Jawa Timur,HOTEL,2.0,437789.420298,8.572257,16.00000
4,Chonburi,GUESTHOUSE,2.0,212348.750150,8.017347,16.00000


In [69]:
y_quantitative = ["next_accom_star_rating","next_selling_price","next_hotel_score","next_room_size_inmeter"]
y_qualitative  = ["next_accom_region","next_accom_type"]


def build_min_max_scaler(df) :
  scaled_columns = pd.DataFrame()
  for i in df.columns:
      min = df[i].min()
      df[i] = df[i]-min
      max = df[i].max()
      scaled_columns[i] = pd.DataFrame( round(df[i]/max ,5) )

      # print(i, max)
      # print(i, min)
      print("")
  return scaled_columns

y_quantitative = build_min_max_scaler(df[y_quantitative])

y_final = y_quantitative 
y_final["next_accom_region"] = y["next_accom_region"]
y_final["next_accom_type"] = y["next_accom_type"]
y_final.head()

<ipython-input-69-0b1ad83ee884>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = df[i]-min
<ipython-input-69-0b1ad83ee884>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = df[i]-min
<ipython-input-69-0b1ad83ee884>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

,next_accom_star_rating,next_selling_price,next_hotel_score,next_room_size_inmeter,next_accom_region,next_accom_type
0,0.6,0.02719,0.52637,0.00694,Jawa Timur,HOMESTAY
1,0.8,0.02512,0.80081,0.00694,Jawa Barat,HOTEL
2,0.4,0.00835,0.61996,0.00514,Trang,HOTEL
3,0.6,0.01000,0.73483,0.00722,Jawa Timur,HOTEL
4,0.2,0.01233,0.46296,0.00444,Chonburi,GUESTHOUSE


In [70]:
categorical_columns = [4,5]
kp = KPrototypes(n_clusters=2, init='Cao', n_init=1, verbose=2)
clusters = kp.fit_predict(y_final, categorical=categorical_columns)


Initialization method and algorithm are deterministic. Setting n_init to 1.


ValueError: ignored

In [73]:
y_final.isnull().sum()

next_accom_star_rating        0
next_selling_price            0
next_hotel_score              0
next_room_size_inmeter        0
next_accom_region         11364
next_accom_type           11364
dtype: int64